In [1]:
!pip install pycountry
!pip install langdetect
!pip install transliterate
!pip install textdistance

You should consider upgrading via the '/home/droman/Documents/stuff/dl_course/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/droman/Documents/stuff/dl_course/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/droman/Documents/stuff/dl_course/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/droman/Documents/stuff/dl_course/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

from sklearn.model_selection import train_test_split

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

%pylab inline
plt.style.use("bmh")
import pandas as pd
import numpy as np
from multiprocessing import  Pool

from tqdm import tqdm
tqdm.pandas()
from functools import reduce

import pycountry
import re

import langdetect
from transliterate import translit, get_available_language_codes, slugify

import textdistance as dist


[nltk_data] Downloading package wordnet to /home/droman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/droman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/droman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/droman/Documents/stuff/dl_course/venv/lib/python3.8/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Populating the interactive namespace from numpy and matplotlib


In [3]:
def get_popular_words(data):
    documents = pd.concat([data['name_1'], data['name_2']]).values.astype('U')
    count_vectorizer = CountVectorizer(max_features=1000) # memory limit: ngram_range=(2, 5),
    values = count_vectorizer.fit_transform(documents)
    frequency = values.toarray().sum(axis=0)

    feature_names = count_vectorizer.get_feature_names()
    count_data = pd.DataFrame({'feature_names': feature_names, 'frequency': frequency})


    tfidf_vectorizer = TfidfVectorizer(max_features=1000) # memory limit: ngram_range=(2, 5),
    values = tfidf_vectorizer.fit_transform(documents)
    frequency = values.toarray().sum(axis=0)

    feature_names = tfidf_vectorizer.get_feature_names()
    tf_idf_data = pd.DataFrame({'feature_names': feature_names, 'frequency': frequency})


    tf_idf_data = tf_idf_data.sort_values(by=['frequency'], ascending=False)
    count_data = count_data.sort_values(by=['frequency'], ascending=False)

    perc = 0.2
    size = int(len(count_data) * perc)

    count_stop = count_data['feature_names'].iloc[:size].values
    tf_idf_stop = tf_idf_data['feature_names'].iloc[:size].values

    popular_words = set(count_stop) | set(tf_idf_stop)
    return popular_words

def transliterate_text(text):
    try:
        t = slugify(text)
    except:
        return text
    return t if t is not None else text

def clear_data(data, is_train=True):
    # cities = pd.read_csv("../cities.csv", header=None, encoding='iso-8859-1')

    # drop incorrect empty values
    if is_train:
        data = data.drop_duplicates()
        data = data.dropna()
        data = data[data['name_1'].astype(bool) & data['name_2'].astype(bool)]

    data['name_1'] = data['name_1'].apply(transliterate_text)
    data['name_2'] = data['name_2'].apply(transliterate_text)

    countries = [country.name.lower() for country in pycountry.countries]

    data["name_1"] = data["name_1"].str.lower()
    data["name_2"] = data["name_2"].str.lower()

    # REMOVE COMPANIES LABELS
    legal_entities = ["ltd.", "co.", "inc.", "b.v.", "s.c.r.l.", "gmbh", "pvt."]

    for entity in tqdm(legal_entities):
        data.replace(re.compile(f"\s+{entity}\s*"), "", inplace=True)

    # REMOVE DATA IN CORNERS
    data.replace(re.compile(r"\s+\(.*\)"), "", inplace=True)
    data.replace(re.compile(r"[^\w\s]"), "", inplace=True)

    # REMOVE COUNTRY WORDS
    for country in tqdm(countries):
        data.replace(re.compile(country), "", inplace=True)

    stop_words = stopwords.words("english")

    print("Start remmoving stop words")
    data['name_1'] = data['name_1'].apply(lambda sentence: ' '.join([w for w in sentence.rstrip().split() if not w in stop_words]))
    data['name_2'] = data['name_2'].apply(lambda sentence: ' '.join([w for w in sentence.rstrip().split() if not w in stop_words]))

    # LEMMATIZE
    # приводит к нормальной форме
    lemmatizer = WordNetLemmatizer() #  lemmatizer.lemmatize(word)
    # отрезает окончание
    stemmer = PorterStemmer() # stemmer.stem(word)

    print("Start lemmatization")
    data['name_1'] = data['name_1'].apply(lambda sentence: ' '.join([lemmatizer.lemmatize(w) for w in sentence.rstrip().split()]))
    data['name_2'] = data['name_2'].apply(lambda sentence: ' '.join([lemmatizer.lemmatize(w) for w in sentence.rstrip().split()]))

    data.fillna('', inplace=True)
    popular_words = get_popular_words(data)

    print("Start removing popular words")
    data['name_1'] = data['name_1'].apply(lambda s: ' '.join([w for w in s.rstrip().split() if not w in popular_words]))
    data['name_2'] = data['name_2'].apply(lambda s: ' '.join([w for w in s.rstrip().split() if not w in popular_words]))

    # drop incorrect empty values
    if is_train:
        data = data.drop_duplicates()
        data = data.dropna()
        data = data[data['name_1'].astype(bool) & data['name_2'].astype(bool)]
    return data

LIST_UNIQUE_COMPANIES_ID = 0
DICT_OF_RELATIONS_ID = 1
COUNT_OF_RELATIONS_ID = 2
EXTRA_RELATIONS_ID = 3

def get_same_companies_graph(companies_dataframe):
    result = [] # [[[a,b,c], {a: [b, c], b: [c]}, 3], [[d,e], {d: [e]}], 2]
    for i, pair in tqdm(companies_dataframe.iterrows()):
        first = pair['name_1']
        second = pair['name_2']
        if first != second:
            flag = True

            for list_same_companies in result:
                if first in list_same_companies[0]:
                    if second not in list_same_companies[0]:
                        list_same_companies[0].append(second)
                        try:
                            list_same_companies[1][first].append(second)
                        except:
                            list_same_companies[1].update({first: [second]})
                        list_same_companies[2]+=1
                        flag = False
                else:
                    if second in list_same_companies[0]:
                        if first not in list_same_companies[0]:
                            list_same_companies[0].append(first)
                            try:
                                list_same_companies[1][second].append(first)
                            except:
                                list_same_companies[1].update({second: [first]})
                            list_same_companies[2]+=1
                            flag = False
            if flag:
                result.append([[first, second], {first: [second]}, 1])
    return result

def get_extra_relations(data):
    data = data[data.is_duplicate==1]
    data = data[['name_1','name_2']]

    companies_graph = get_same_companies_graph(data)

    def get_extra_relations(inner_companies_graph):
        extra_relations = []
        for unique_companies in tqdm(inner_companies_graph):
            unique_companies.append([])
            companies = unique_companies[LIST_UNIQUE_COMPANIES_ID]
            company_relations = unique_companies[DICT_OF_RELATIONS_ID]
            for company in companies:
                try:
                    relations = company_relations[company]
                    excluded_companies = relations
                except:
                    excluded_companies = []
                excluded_companies.append(company)
                for extra_relation_company in companies:
                    if extra_relation_company not in excluded_companies:
                        extra_relations.append([company, extra_relation_company])
                        unique_companies[COUNT_OF_RELATIONS_ID]+=1
                        unique_companies[EXTRA_RELATIONS_ID].append([company, extra_relation_company])
        return extra_relations

    extra_relations = get_extra_relations(companies_graph)
    df = pd.DataFrame(extra_relations, columns = ['name_1','name_2'])
    df['is_duplicate'] =   1
    return df, companies_graph

def get_abbreviation(data):
    data = data[data.is_duplicate==0]

    data = data['name_1'].append(data['name_2']).reset_index(drop=True)
    def filter_condition(sentence):
        words = sentence.split()
        return 2 < len(words) < 6 and all(len(word)>3 for word in words)

    data = list(filter(filter_condition, data.tolist()))

    def _get_abbreviation(sentence):
        return ''.join([word[0] for word in sentence.split()])
    data = [[sentence, _get_abbreviation(sentence)] for sentence in data]

    return pd.DataFrame(data, columns=['name_1', 'name_2'])


def normalize(values):
    return (values - values.min()) / (values.max() - values.min())

def calculate_distance(df, col_1="name_1", col_2="name_2", prefix=""):
    print("First part of distance finished.")
    df[prefix + "jaccard"] = df.apply(lambda r: dist.jaccard(r[col_1], r[col_2]), axis=1)
    df[prefix + "cosine"]  = df.apply(lambda r: dist.cosine(r[col_1], r[col_2]), axis=1)
    df[prefix + "mra"]     = df.apply(lambda r: dist.mra(r[col_1], r[col_2]), axis=1)

    print("Second part of distance finished.")
    dist.jaro_winkler.external = False
    df[prefix + "jaro_winkler"] = df.apply(lambda r: dist.jaro_winkler(r[col_1], r[col_2]), axis=1)

    df[prefix + "hamming"] = df.apply(lambda r: dist.hamming(r[col_1], r[col_2]), axis=1)
    df[prefix + "mlipns"]  = df.apply(lambda r: dist.mlipns(r[col_1], r[col_2]), axis=1)
    df[prefix + "damerau_levenshtein"] = df.apply(lambda r: dist.damerau_levenshtein(r[col_1], r[col_2]), axis=1)
    df[prefix + "strcmp95"] = df.apply(lambda r: dist.strcmp95(r[col_1], r[col_2]), axis=1)
    df[prefix + "needleman_wunsch"] = df.apply(lambda r: dist.needleman_wunsch(r[col_1], r[col_2]), axis=1)

    print("Third part of distance finished.")
    # error # df[prefix + "gotoh"] = df.apply(lambda r: dist.gotoh(r[col_1], r[col_2]), axis=1)
    df[prefix + "sorensen"] = df.apply(lambda r: dist.sorensen(r[col_1], r[col_2]), axis=1)
    df[prefix + "tversky"]  = df.apply(lambda r: dist.tversky(r[col_1], r[col_2]), axis=1)
    df[prefix + "overlap"]  = df.apply(lambda r: dist.overlap(r[col_1], r[col_2]), axis=1)
    #  many inf #df[prefix + "tanimoto"] = df.apply(lambda r: dist.tanimoto(r[col_1], r[col_2]), axis=1)
    df[prefix + "monge_elkan"] = df.apply(lambda r: dist.monge_elkan(r[col_1], r[col_2]), axis=1)

    print("Fourth part of distance finished.")
    df[prefix + "bag"]    = df.apply(lambda r: dist.bag(r[col_1], r[col_2]), axis=1)
    df[prefix + "lcsseq"] = df.apply(lambda r: dist.lcsseq(r[col_1], r[col_2]), axis=1)
    df[prefix + "lcsstr"] = df.apply(lambda r: dist.lcsstr(r[col_1], r[col_2]), axis=1)
    df[prefix + "ratcliff_obershelp"] = df.apply(lambda r: dist.ratcliff_obershelp(r[col_1], r[col_2]), axis=1)
    return df

def get_split_index(inner_companies_graph, input_percent=20):
    PERCENT_OF_COMPANIES_ID = 3
    percent_count = 0
    for i in range(len(inner_companies_graph)-1, -1, -1):
        if percent_count >= input_percent:
            return i
        percent_count+=inner_companies_graph[i][PERCENT_OF_COMPANIES_ID]

def get_splited_data(companies_graph, index):
    train_graph = companies_graph[:index]
    val_graph = companies_graph[index:]
    def flat_graph_companies(graph_data):
        def flat_one_company_dict(company_dict):
            key = list(company_dict.items())[0][0]
            values = list(company_dict.items())[0][1]
            result = []
            for company in values:
                result.append([key, company])
            return result

        graph_list = list(map(lambda x: flat_one_company_dict(x[DICT_OF_RELATIONS_ID]), graph_data))
        return [item for sublist in graph_list for item in sublist]
    train_data = flat_graph_companies(train_graph)
    val_data = flat_graph_companies(val_graph)
    return train_data, val_data

def split_duplicate_data(data, companies_graph, percent=20):
    duplicate_data = data[data["is_duplicate"]==1]

    companies_graph.sort(key=lambda x: x[COUNT_OF_RELATIONS_ID])
    total_companies_number = reduce(lambda x1, x2: x1+x2, map(lambda x: x[COUNT_OF_RELATIONS_ID], companies_graph))
    for company in companies_graph:
        company.append(100*(company[COUNT_OF_RELATIONS_ID]/total_companies_number))
    split_index = get_split_index(companies_graph, percent)
    train_data, val_data = get_splited_data(companies_graph, split_index)
    train_data = pd.DataFrame(train_data, columns=["name_1", "name_2"])
    val_data = pd.DataFrame(val_data, columns=["name_1", "name_2"])
    train_data["is_duplicate"] = 1
    val_data["is_duplicate"] = 1
    return train_data, val_data

def split_not_duplicate_data(data, percent=20):
    duplicate_data = data[data["is_duplicate"]==0]
    data_train, data_val = train_test_split(duplicate_data, test_size=percent, random_state=42)
    return data_train, data_val

def parallelize_dataframe(df, func, n_cores=25):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def parallel_clear_data(data):
    return parallelize_dataframe(data, clear_data)


def parallel_calculate_distance(data):
    return parallelize_dataframe(data, calculate_distance)

In [4]:
train = pd.read_csv("./data/train.csv")
wiki_data = pd.read_csv("./data/wiki_data.csv")

In [5]:
print(len(train))
print(len(wiki_data))

497819
6011


In [6]:
wiki_data['is_duplicate'] = 1
train = train.drop(['pair_id'], axis=1)
train = pd.concat([train, wiki_data])

In [7]:
print(len(train))
train.head()

503830


,name_1,name_2,is_duplicate
0,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,Powermax Rubber Factory,Co. One,0
4,Tress A/S,Longyou Industries Park Zhejiang,0


In [8]:
data = parallel_clear_data(train)
data.to_csv("data/clear_data.csv")
print("Clearing finished")

extra_relations, companies_graph = get_extra_relations(data)
extra_relations['is_duplicate'] = 1

abbreviations = get_abbreviation(data)
abbreviations['is_duplicate'] = 1

data_tmp = pd.concat([data, extra_relations, abbreviations])
data_tmp.to_csv("data/extra_clear_data.csv")
print("Getting extra data finished")

data = pd.concat([data, abbreviations])
train_duplicate_data, val_duplicate_data = split_duplicate_data(extra_relations, companies_graph, 20)
train_not_duplicate_data, val_not_duplicate_data = split_not_duplicate_data(data, 20)

train = pd.concat([train_duplicate_data, train_not_duplicate_data])
val = pd.concat([val_duplicate_data, val_not_duplicate_data])


train.to_csv("data/train_split_data.csv")
val.to_csv("data/val_split_data.csv")
print("Spliting data finished")

train_with_distance = parallel_calculate_distance(train)
val_with_distance = parallel_calculate_distance(val)

train_with_distance.to_csv("data/train_distance_data.csv")
val_with_distance.to_csv("data/val_distance_data.csv")
print("Calculating distance finished")

100%|██████████| 7/7 [00:04<00:00,  1.66it/s]

100%|██████████| 7/7 [00:04<00:00,  1.69it/s]s]

100%|█████████▉| 248/249 [01:31<00:00,  2.93it/s]

100%|██████████| 249/249 [01:33<00:00,  2.66it/s]

 98%|█████████▊| 245/249 [01:31<00:01,  2.51it/s]

100%|██████████| 249/249 [01:32<00:00,  2.68it/s]
5717it [00:01, 4341.25it/s]
100%|██████████| 1271/1271 [00:00<00:00, 3127.10it/s]
266120it [33:48, 131.20it/s]


Start remmoving stop words
Start remmoving stop words
Start lemmatization
Start lemmatization
Start remmoving stop words
Start remmoving stop words
Start remmoving stop words
Start remmoving stop words
Start remmoving stop words
Start remmoving stop words
Start remmoving stop words
Start remmoving stop wordsStart remmoving stop words

Start remmoving stop words
Start remmoving stop words
Start remmoving stop wordsStart remmoving stop wordsStart remmoving stop words


Start remmoving stop words
Start remmoving stop words
Start remmoving stop words
Start remmoving stop wordsStart remmoving stop words

Start lemmatization
Start lemmatizationStart lemmatization

Start remmoving stop words
Start remmoving stop words
Start lemmatization
Start lemmatization
Start lemmatization
Start remmoving stop wordsStart lemmatization

Start remmoving stop words
Start lemmatization
Start lemmatization
Start lemmatization
Start lemmatizationStart lemmatization

Start lemmatizationStart lemmatization

Start

In [9]:
train_with_distance.head()

,name_1,name_2,is_duplicate,jaccard,cosine,mra,jaro_winkler,hamming,mlipns,damerau_levenshtein,strcmp95,needleman_wunsch,sorensen,tversky,overlap,monge_elkan,bag,lcsseq,lcsstr,ratcliff_obershelp
0,carlisletings waterproofing,carlisletings wtrprfng,1,0.814815,0.902671,6,0.962963,11,0,5,0.962963,17.0,0.897959,0.814815,1.0,0.034294,5,carlisletings wtrprfng,carlisletings w,0.897959
1,ooohiminvestgrupp,himinvestgruppooo,1,1.000000,1.000000,3,0.882353,17,0,6,0.882353,8.0,1.000000,1.000000,1.0,0.058824,0,himinvestgrupp,himinvestgrupp,0.823529
2,soudal,ooosoudal,1,0.666667,0.816497,0,0.833333,8,0,3,0.833333,3.0,0.800000,0.666667,1.0,0.166667,3,soudal,soudal,0.800000
3,midland inc,midland,1,0.636364,0.797724,3,0.927273,4,1,4,0.927273,3.0,0.777778,0.636364,1.0,0.074380,4,midland,midland,0.777778
4,gate,gatesporation,1,0.307692,0.554700,0,0.861538,9,0,9,0.861538,-5.0,0.470588,0.307692,1.0,0.250000,9,gate,gate,0.470588
